# Predicting house prices using k-nearest neighbors regression
In this notebook, you will implement k-nearest neighbors regression. You will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

## Load

In [181]:
import sklearn
import pandas

## Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [182]:
small_data = pandas.read_csv("kc_house_data_small.csv", index_col=0)

## Import useful functions from previous notebooks

As in Exercise 1 (*exercise-1.ipynb*), we convert the DataFrame into a 2D Numpy array. Copy and paste `get_numpy_data()` from the exercise there.

In [183]:
import numpy as np # note this allows us to refer to numpy as np instead

In [184]:
def get_numpy_data(data, features_title, output_title):
    if('constant' not in data):
        data['constant'] = 1
    features_title = ['constant'] + features_title
    features_columns = data[features_title]
    feature_matrix = features_columns.values
    output_column = data[output_title]
    output_array = output_column.values
    return(feature_matrix, output_array)

We will also need the `normalize_features()` function from the prior Exercise 2 (*exercise-2.ipynb*) to normalizes all feature columns to unit norm. Paste it into the cell below.

In [185]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)

## Split data into training, test, and validation sets

In [186]:
from sklearn.model_selection import train_test_split
train_and_validation, test_data = train_test_split(small_data, train_size=0.9, test_size=0.1, random_state=1)
train_data, validation_data = train_test_split(train_and_validation, train_size=0.5, test_size=0.5, random_state=1)
train_data = train_data.copy()
test_data = test_data.copy()
validation_data = validation_data.copy()

## Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transform the training, test, and validation SFrames into Numpy arrays:

In [187]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train_data, feature_list, 'price')
features_test, output_test = get_numpy_data(test_data, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation_data, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [188]:
features_train, norms = normalize_features(features_train) # normalize training set features (columns)
features_test = features_test / norms # normalize test set by training set norms
features_valid = features_valid / norms # normalize validation set by training set norms

# Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1.

In [189]:
query_point = features_test[0]

Now print the 10th row (index 9) of the training feature matrix. Again, you get an 18-dimensional vector with components between 0 and 1.

In [190]:
features_train[9]

array([ 0.01598007,  0.01364581,  0.01244476,  0.01114439,  0.00332368,
        0.01516487,  0.        ,  0.        ,  0.01845494,  0.0123755 ,
        0.01292299,  0.        ,  0.01583581,  0.        ,  0.0160461 ,
       -0.01599405,  0.0136938 ,  0.0049883 ])

***QUIZ QUESTION ***

What is the Euclidean distance between the query house and the 25th house of the training set? 

Note: Do not use the `np.linalg.norm` function; use `np.sqrt`, `np.sum`, and the power operator (`**`) instead. The latter approach is more easily adapted to computing multiple distances at once. Actually, not really, but you should know things under the hood.

In [191]:
house_25th = features_train[24]

In [192]:
Eu_distance = np.sqrt(np.sum((query_point - house_25th)**2))

In [193]:
Eu_distance
round(0.041624810106566046, 3)

0.042

# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

In [194]:
squared_distance = (features_train - query_point)**2
#print squared_distance
sum_distance = np.empty(10)
for i in range(10):
    sum_distance[i] = np.sum(squared_distance[i,:])
#print sum_distance
educlidean_distance = np.sqrt(sum_distance)
print (educlidean_distance)

[0.03935029 0.02551734 0.02642999 0.01212119 0.03278956 0.01418313
 0.01301056 0.01253362 0.01553064 0.00973326]


*** QUIZ QUESTION ***

Among the first 10 training houses, which house is the closest to the query house?

In [195]:
np.argmin(Eu_d, axis=0)

0

It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [196]:
for i in range(3):
    print(features_train[i]-features_test[0])
    # should print 3 vectors of length 18

[ 0.00000000e+00  4.54860395e-03  8.88911716e-03  1.01631240e-02
  5.28008301e-03 -1.01099109e-02  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.60085326e-03  3.34503100e-02
 -1.62169111e-04  0.00000000e+00  4.80166661e-05  1.09833955e-05
  8.82489138e-03  3.43005338e-03]
[ 0.00000000e+00  1.36458118e-02  1.06669406e-02  9.39212839e-03
  3.40508097e-03 -1.01099109e-02  0.00000000e+00  0.00000000e+00
  4.61373598e-03  0.00000000e+00  1.08910730e-02  0.00000000e+00
 -1.70277567e-04  0.00000000e+00 -1.27820432e-04  4.05339597e-05
  1.06507310e-03  2.83595753e-03]
[ 0.00000000e+00  4.54860395e-03  1.06669406e-02  9.88276196e-03
  3.01531785e-03 -1.01099109e-02  0.00000000e+00  0.00000000e+00
  9.22747195e-03  4.12516718e-03  1.14600097e-02  0.00000000e+00
 -3.32446678e-04  0.00000000e+00 -1.34406343e-06  7.45301840e-06
  1.01181944e-02  4.37220538e-03]


The subtraction operator (`-`) in Numpy is vectorized as follows:

In [197]:
print(features_train[0:3] - features_test[0])

[[ 0.00000000e+00  4.54860395e-03  8.88911716e-03  1.01631240e-02
   5.28008301e-03 -1.01099109e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.60085326e-03  3.34503100e-02
  -1.62169111e-04  0.00000000e+00  4.80166661e-05  1.09833955e-05
   8.82489138e-03  3.43005338e-03]
 [ 0.00000000e+00  1.36458118e-02  1.06669406e-02  9.39212839e-03
   3.40508097e-03 -1.01099109e-02  0.00000000e+00  0.00000000e+00
   4.61373598e-03  0.00000000e+00  1.08910730e-02  0.00000000e+00
  -1.70277567e-04  0.00000000e+00 -1.27820432e-04  4.05339597e-05
   1.06507310e-03  2.83595753e-03]
 [ 0.00000000e+00  4.54860395e-03  1.06669406e-02  9.88276196e-03
   3.01531785e-03 -1.01099109e-02  0.00000000e+00  0.00000000e+00
   9.22747195e-03  4.12516718e-03  1.14600097e-02  0.00000000e+00
  -3.32446678e-04  0.00000000e+00 -1.34406343e-06  7.45301840e-06
   1.01181944e-02  4.37220538e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [198]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print(results[0] - (features_train[0]-features_test[0]))
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print(results[1] - (features_train[1]-features_test[0]))
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print(results[2] - (features_train[2]-features_test[0]))
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Aside: it is a good idea to write tests like this cell whenever you are vectorizing a complicated operation.

# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house.

In [199]:
diff = features_train[0:len(features_train)] - query_point

In [200]:
diff

array([[ 0.00000000e+00,  4.54860395e-03,  8.88911716e-03, ...,
         1.09833955e-05,  8.82489138e-03,  3.43005338e-03],
       [ 0.00000000e+00,  1.36458118e-02,  1.06669406e-02, ...,
         4.05339597e-05,  1.06507310e-03,  2.83595753e-03],
       [ 0.00000000e+00,  4.54860395e-03,  1.06669406e-02, ...,
         7.45301840e-06,  1.01181944e-02,  4.37220538e-03],
       ...,
       [ 0.00000000e+00,  0.00000000e+00, -3.55564686e-03, ...,
        -4.70716952e-06,  1.44545635e-03,  2.66192946e-03],
       [ 0.00000000e+00,  0.00000000e+00,  1.77782343e-03, ...,
        -7.45301840e-06,  3.87990914e-03,  1.00166160e-03],
       [ 0.00000000e+00,  4.54860395e-03,  1.77782343e-03, ...,
         1.24216973e-05,  1.68129396e-02,  3.90312971e-03]])

To test the code above, run the following cell, which should output a value 0.028905448402567344:

In [201]:
print(diff[-1].sum()) # sum of the feature differences between the query and last training house
# should print 0.028905448402567344

0.028905448402567344


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [202]:
print(np.sum(diff**2, axis=1)[15]) 

0.0002609401294849807


In [203]:
np.sum(diff**2, axis=1)

array([0.00154845, 0.00065113, 0.00069854, ..., 0.00013687, 0.00017795,
       0.00046414])

In [204]:
print(np.sum(diff[15]**2))

0.0002609401294849807


With this result in mind, write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Assign the result to a variable `distances`.

**Hint**: Do not forget to take the square root of the sum of squares.

In [205]:
distances = np.sqrt(np.sum(diff**2, axis=1))

In [206]:
distances

array([0.03935029, 0.02551734, 0.02642999, ..., 0.01169909, 0.01333964,
       0.02154386])

To test the code above, run the following cell, which should output a value 0.0125217497650:

In [207]:
print(distances[100]) # Euclidean distance between the query house and the 101th training house
# should print 0.0125217497650

0.012521749765023885


Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [208]:
def mass_distance(query_point, training_features):
    diff = training_features - query_point
    distances = np.sqrt(np.sum(diff**2, axis=1))
    return (distances)

*** QUIZ QUESTIONS ***

1.  Take the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [209]:
query_point = features_test[2]

In [210]:
distances = mass_distance(query_point, features_train)
np.argmin(distances, axis=0)

630

In [211]:
output_train[630]

839990

# Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [212]:
def indices_of_KNN(k,features_train,query_point):
    distance = mass_distance(features_train,query_point)
    sorted_distance = np.argsort(distance)
    indice = sorted_distance[0:k]
    return(indice)

*** QUIZ QUESTION ***

Take the query house to be third house of the test set (`features_test[2]`).  What are the indices of the 4 training houses closest to the query house?

In [213]:
query_house = features_test[2]
k = 4

In [214]:
indice = indices_of_KNN(k, features_train, query_house)
print (indice)

[ 630 2631   62 3641]


## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**Hint**: You can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.

In [215]:
def k_nearest_avg(k, query_point, features_train, output_train):
    indice = indices_of_KNN(k, features_train, query_point)
    predicted_value = np.mean(output_train[indice])
    return (predicted_value)

*** QUIZ QUESTION ***

Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

In [216]:
predicted_value = k_nearest_avg(4, features_test[2], features_train, output_train)
print(predicted_value)

835870.0


In [217]:
predicted_value = k_nearest_avg(1, features_test[2], features_train, output_train)
print(predicted_value)

839990.0


Compare this predicted value using 4-nearest neighbors to the predicted value using 1-nearest neighbor computed earlier.

## Make multiple predictions

Write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function should return a set of predicted values, one for each house in the query set.

**Hint**: To get the number of houses in the query set, use the `.shape` field of the query features matrix. See [the documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [218]:
def multiple_prediction_KNN(k, features_train, output_train, features_query):
    num_features_query = features_query.shape[0]
    print(num_features_query)
    predicted_query = np.empty(num_features_query) #don't initial elements
    for i in range(num_features_query):
        indice = indices_of_KNN(k, features_train, features_query[i])
        predicted_query[i] = np.mean(output_train[indice])
    return(predicted_query)

*** QUIZ QUESTION ***

Make predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`. 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

In [219]:
predicted_value = multiple_prediction_KNN(10, features_train, output_train, features_test[0:10])

10


In [220]:
predicted_value

array([391150. , 464009.5, 803453.1, 409795. , 452100. , 350495. ,
       462995. , 312840. , 850584.1, 517227.7])

In [221]:
predicted_value.min()

312840.0

## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

(Depending on your computing environment, this computation may take 10-15 minutes.)

In [ ]:
rss_all = np.empty(15)
for k in range(1,16):
    print(k)
    predicted_values = multiple_prediction_KNN(k,features_train,output_train,features_valid)
    error = predicted_values - output_valid
    rss_all[k-1] = np.dot(error,error)
print(rss_all)

1
3916
2
3916
3
3916
4
3916
5
3916
6
3916
7
3916
8
3916
9
3916
10
3916
11
3916
12
3916
13
3916


In [ ]:
min(rss_all)

To visualize the performance as a function of `k`, plot the RSS on the VALIDATION set for each considered `k` value:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

k_value = range(1,16)
plt.plot(k_value, rss_all, 'bo-')

***QUIZ QUESTION***

What is the RSS on the TEST data using the value of k found above?  To be clear, sum over all houses in the TEST set.

In [ ]:
predicted_values = multiple_prediction_KNN(8, features_train, output_train, features_test)
error = predicted_values - output_test
print(np.dot(error, error))

In [ ]:
round(71490771425837.53)